In [ ]:
# packages
import os
import sys
import scipy
import scipy.io as sio
import scanpy.external as sce
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import umap.umap_ as umap
import re
import anndata
import scirpy as ir
from matplotlib.gridspec import GridSpec
import random
import scipy.stats as stats
from scipy.stats import median_abs_deviation

In [ ]:
## T Cells

import gseapy
import gseapy as gp

tcell_data = adata[adata.obs["pie_clustering"].isin(["T Cells"])]
tcell_data.uns['log1p']["base"] = None

from gseapy import Msigdb

sc.tl.rank_genes_groups(tcell_data,
                        groupby='type',
                        use_raw=True,
                        method='wilcoxon',
                        groups=["sorted"],
                        reference='unsorted')

sc.pl.rank_genes_groups(tcell_data, n_genes=25, sharey=False)

In [ ]:
#see full list of latest enrichr library names, which will pass to -g parameter:
names = gseapy.get_library_name()

print(names)

In [ ]:
# get deg result
result = tcell_data.uns['rank_genes_groups']
groups = result['names'].dtype.names
degs = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})

# subset up or down regulated genes
degs_sig = degs[degs.sorted_pvals_adj < 0.05]
degs_up = degs_sig[degs_sig.sorted_logfoldchanges > 0]
degs_dw = degs_sig[degs_sig.sorted_logfoldchanges < 0]

# Enricr API
enr_up = gp.enrichr(degs_up.sorted_names,
                    gene_sets='MSigDB_Hallmark_2020',
                    outdir=None)

enr_dw = gp.enrichr(degs_dw.sorted_names,
                    gene_sets='MSigDB_Hallmark_2020',
                    outdir=None)

# concat results
enr_up.res2d['UP_DW'] = "UP"
enr_dw.res2d['UP_DW'] = "DOWN"
enr_res = pd.concat([enr_up.res2d.head(10), enr_dw.res2d.head(10)])

from gseapy.scipalette import SciPalette
sci = SciPalette()
NbDr = sci.create_colormap()
# NbDr

ax = gp.barplot(enr_res, figsize=(3,5),
                group ='UP_DW',
                title ="MSig DB",
                color = ['b','r'])
ax.set_title("T Cells")

In [ ]:
# repeat above for macro/mono cells

In [ ]:
## Macro/Mono Cells

import gseapy

mmcell_data = adata[adata.obs["pie_clustering"].isin(["Macro./Mono."])]
mmcell_data.uns['log1p']["base"] = None

from gseapy import Msigdb

sc.tl.rank_genes_groups(mmcell_data,
                        groupby='type',
                        use_raw=True,
                        method='wilcoxon',
                        groups=["sorted"],
                        reference='unsorted')

sc.pl.rank_genes_groups(mmcell_data, n_genes=25, sharey=False)

In [ ]:
# get deg result
result = mmcell_data.uns['rank_genes_groups']
groups = result['names'].dtype.names
degs = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})

# subset up or down regulated genes
degs_sig = degs[degs.sorted_pvals_adj < 0.05]
degs_up = degs_sig[degs_sig.sorted_logfoldchanges > 0]
degs_dw = degs_sig[degs_sig.sorted_logfoldchanges < 0]

# Enricr API
enr_up = gp.enrichr(degs_up.sorted_names,
                    gene_sets='MSigDB_Hallmark_2020',
                    outdir=None)

enr_dw = gp.enrichr(degs_dw.sorted_names,
                    gene_sets='MSigDB_Hallmark_2020',
                    outdir=None)

# concat results
enr_up.res2d['UP_DW'] = "UP"
enr_dw.res2d['UP_DW'] = "DOWN"
enr_res = pd.concat([enr_up.res2d.head(10), enr_dw.res2d.head(10)])

from gseapy.scipalette import SciPalette
sci = SciPalette()
NbDr = sci.create_colormap()
# NbDr

ax = gp.barplot(enr_res, figsize=(3,5),
                group ='UP_DW',
                title ="MSig DB",
                color = ['b','r'])
ax.set_title("Macro./Mono.")

In [ ]:
# spot check

for i, r, in enr_res.iterrows():
    genes = r["Genes"].split(";")
    direction = r["UP_DW"]
    sc.pl.dotplot(mmcell_data,genes, groupby="type", standard_scale="var",  title=direction)